In [1]:
import pandas as pd
from tqdm import tqdm
from itables import show as table_show

In [15]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [3]:
# obtention des donnnées de la CAN :
directory = '~/Bureau/Exports/20240513/non_anonyme/'
file = 'AGS_20240513_exports_agronomes_interventions_synthetisees.csv'
df_can = pd.read_csv(directory+file, sep='@')

In [32]:
df_studied = df_can.sample(10)

In [43]:
df_lutte_bio = df_can.loc[
    ~df_can['proportion_surface_traitee_lutte_bio'].isna()
].sample(2)

In [44]:
df_studied = pd.concat([df_studied, df_lutte_bio])

In [3]:
df_studied[['intervention_id', 'interventions_actions', 'proportion_surface_traitee_phyto', 'psci_phyto', 'proportion_surface_traitee_lutte_bio', 'psci_lutte_bio', 'quantite_eau_mm']]

NameError: name 'df_studied' is not defined

In [4]:
table_show(pd.melt(df_studied[['intervention_id',  'interventions_actions', 'proportion_surface_traitee_phyto', 'psci_phyto', 'proportion_surface_traitee_lutte_bio', 'psci_lutte_bio', 'quantite_eau_mm', 'precedent_nom', 'precedent_especes_edi']], id_vars=['intervention_id']))

NameError: name 'df_studied' is not defined

In [5]:
print(list(df_studied['intervention_id'].values))

NameError: name 'df_studied' is not defined

In [6]:
donnees = {}

def import_df(df_name, path_data, sep, index_col=None):
    donnees[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'intervention_synthetise', 'action_synthetise', 'fertilisation_organique'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

100%|██████████| 2/2 [00:10<00:00,  5.20s/it]


In [27]:
import_dfs(['utilisation_intrant_synthetise'], ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)


100%|██████████| 1/1 [00:06<00:00,  6.04s/it]


In [29]:
import_dfs(['intrant'], ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)


100%|██████████| 1/1 [00:04<00:00,  4.04s/it]


In [7]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_79c35d59-052d-4924-9360-90d77b5ad9c3', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_d95eafc1-b3cd-4a26-bed9-81ebac609873', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_a3da83ef-9c4d-4dcf-84fb-40f91fe6a9d3', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_6bf87183-b4e6-4392-bf02-fb39c09b5b1b', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_7a3d5039-f980-4168-be21-4fb5d526ae70', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_4b7bc033-28e0-4ae3-a487-e7064060202e', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_d372b219-629b-4387-a275-6ada7c994f2f', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_1d39436d-14f7-4388-aaf7-2f408b8dd398', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_6c84fc3b-a199-4e02-9d99-f75613e94be0', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_2a23f87b-d949-4af6-8ab1-f9b6d38d5ce6', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_c2430604-81ac-467d-a7bb-93384909ff6d', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_fa58158b-dac4-4428-a282-3656c7742e97', 
    'fr.inra.agrosyst.api.entities.practiced.PracticedIntervention_50a5aa52-bf54-4912-a1a2-af49b5a6d6e1'
]

In [20]:
donnees['intervention_synthetise_test'] = donnees['intervention_synthetise'].loc[
    donnees['intervention_synthetise'].index.isin(studied_ids)
]
donnees['action_synthetise_test'] = donnees['action_synthetise'].loc[
    donnees['action_synthetise']['intervention_synthetise_id'].isin(studied_ids)
]
donnees['utilisation_intrant_synthetise_test'] = donnees['utilisation_intrant_synthetise'].loc[
    donnees['utilisation_intrant_synthetise']['action_synthetise_id'].isin(donnees['action_synthetise_test'].index)
]
donnees['intrant_test'] = donnees['intrant'].loc[
    donnees['intrant'].index.isin(donnees['utilisation_intrant_synthetise_test']['intrant_id'])
]
donnees['fertilisation_organique_test'] = donnees['fertilisation_organique'].loc[
    donnees['fertilisation_organique']['libelle'].isin(donnees['intrant_test']['ref_nom'])
]

In [44]:
path='./'
donnees['intervention_synthetise_test'].to_csv(path+'intervention_synthetise'+'.csv')
donnees['action_synthetise_test'].to_csv(path+'action_synthetise'+'.csv')
donnees['utilisation_intrant_synthetise_test'].to_csv(path+'utilisation_intrant_synthetise'+'.csv')
donnees['intrant_test'].to_csv(path+'intrant'+'.csv')
donnees['fertilisation_organique_test'].to_csv(path+'fertilisation_organique'+'.csv')